In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains  
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
import time

def fetch_realtime_data():
    # Initialize the webdriver
    chrome_service = Service(r'C:\Users\user\chromedriver-win64\chromedriver.exe')
    driver = webdriver.Chrome(service=chrome_service)
   
    # Open the webpage
    url = "https://epsis.kpx.or.kr/epsisnew/selectEkgeEpsMepRealChart.do?menuId=030300"
    driver.get(url)
    
    # Maximize the window
    driver.maximize_window()
    
    # Scroll to the bottom of the page to load initial data
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)
    
    # Initialize an empty DataFrame to store all data elements
    df = pd.DataFrame(columns=['temporary', 'supply capacity', 'current load', 'supply reserve capacity', 'supply reserve rate'])
    
    # Find the scrollbar element
    scrollbar = driver.find_element(By.CLASS_NAME, 'rMateH5__VBrowserScrollBar')
    
    scroll_count = 0
    max_scrolls = 24  # Set a maximum number of scrolls to prevent infinite loop
    
    while True:
        # Get the initial number of elements
        initial_size = len(df)

        # Get the page source and parse it
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        
        # Find elements containing the data
        elements = soup.find_all(class_=lambda c: c and c.startswith('rMateH5__DataGridItemRenderer rMateH5__DataGridColumn'))
        
        # Extract text from elements and add to a temporary list if not null
        new_elements = [element.text for element in elements if element.text.strip()]
        
        # Get the initial scrollbar position
        initial_scroll_position = driver.execute_script("return arguments[0].scrollTop;", scrollbar)
        
        # Scroll the scrollbar down a small amount
        driver.execute_script("arguments[0].scrollTop += 580;", scrollbar)
        time.sleep(2)
        
        # Get the new scrollbar position
        new_scroll_position = driver.execute_script("return arguments[0].scrollTop;", scrollbar)

        for element in new_elements:
            print(element)
        
        # Split the data into respective categories and append to the DataFrame
        if new_elements:
            num_entries = len(new_elements)
            num_iter = num_entries // 5 * 4   

            supply_capacity = []
            current_load = []
            supply_reserve_capacity = []
            supply_reserve_rate = []
            
            
            for i in range(0,num_iter,4):
                supply_capacity.append(new_elements[i])
                current_load.append(new_elements[i+1])
                supply_reserve_capacity.append(new_elements[i+2])
                supply_reserve_rate.append(new_elements[i+3])
            
            timestamps = new_elements[num_iter:num_entries]
                
            
            
            temp_df = pd.DataFrame({
                'temporary': timestamps,
                'supply capacity': supply_capacity,
                'current load': current_load,
                'supply reserve capacity': supply_reserve_capacity,
                'supply reserve rate': supply_reserve_rate
            })
            
            df = pd.concat([df, temp_df], ignore_index=True)
        
        # Check if scrollbar position did not change
        if initial_scroll_position == new_scroll_position:
            break
        
        # Increment scroll counter and check if max scrolls reached
        scroll_count += 1
        if scroll_count >= max_scrolls:
            print("Reached maximum scroll limit.")
            break
    
    # Close the webdriver
    driver.quit()
    
    # Print the DataFrame
    df = df.drop_duplicates()
    df = df.sort_values(by = 'temporary')

    df.to_excel('today.xlsx', index=False)

In [3]:
fetch_realtime_data()

80,209
59,064
21,146
36
80,126
58,610
21,516
37
80,250
58,459
21,791
37
79,586
58,285
21,301
37
79,691
58,181
21,510
37
79,762
57,768
21,995
38
79,838
57,546
22,291
39
79,935
57,100
22,835
40
80,051
56,751
23,300
41
79,752
56,244
23,508
42
79,852
56,005
23,846
43
79,943
55,452
24,491
44
80,030
55,211
24,820
45
80,148
55,202
24,947
45
80,304
54,668
25,222
46
79,632
54,564
24,657
45
2024-06-09 18:15
2024-06-09 18:10
2024-06-09 18:05
2024-06-09 18:00
2024-06-09 17:55
2024-06-09 17:50
2024-06-09 17:45
2024-06-09 17:40
2024-06-09 17:35
2024-06-09 17:30
2024-06-09 17:25
2024-06-09 17:20
2024-06-09 17:15
2024-06-09 17:10
2024-06-09 17:05
2024-06-09 17:00
79,808
53,898
25,498
47
79,956
53,383
26,160
49
79,994
53,080
26,182
49
79,936
52,676
26,386
49
80,005
52,228
26,653
50
79,992
52,231
26,632
50
80,074
51,995
26,950
51
80,080
51,387
27,564
53
80,111
51,176
27,551
52
80,194
50,789
28,025
54
79,942
50,723
27,835
53
80,037
50,670
27,987
54
80,109
50,470
28,257
55
80,322
50,256
28,683
56
80,422
5